In [1]:
import pandas as pd

from scripts.downloader import DownloaderAktualne, DownloaderIdnes

In [2]:
### Downloading the articles
# Config
url_aktualne = "https://zpravy.aktualne.cz/domaci/"
url_idnes = "https://www.idnes.cz/zpravy/domaci/"

path_aktualne = "data/articles_Aktualne.cz.csv"
path_idnes = "data/articles_iDnes.cz.csv"

In [3]:
# Init verbose DownloaderAktualne and DownloaderIdnes
d_aktualne = DownloaderAktualne(url_aktualne, save_to=path_aktualne, verbose=True)
d_idnes = DownloaderIdnes(url_idnes, save_to=path_idnes, verbose=True)

Succesfully initialized DownloaderAktualne.
Succesfully initialized DownloaderIdnes.


In [18]:
# Download demo
#d_aktualne.downloadHeadlines(from_page = 1, to_page = 5, bulk_size = 40)
d_aktualne.articles_df[100:103]
#d_aktualne.saveToCsv()

#d_idnes.downloadHeadlines(from_page = 1, to_page = 2, bulk_size = 40)
d_idnes.articles_df[100:103]
#d_idnes.saveToCsv()

pd.concat([d_aktualne.articles_df[100:102], d_idnes.articles_df[40:42]])

,article_id,slug,date,time,is_updated,headline,excerpt,article_url,scraped_at
100,r~dacd863c832511eaaabd0cc47ab5f122,jak-se-spojit-s-prateli-sledovat-filmy-zaplati...,2020-05-05,17:19:00,False,"Jak se spojit s přáteli, zaplatit nákup či sle...","Aktuálně.cz nabízí přehled, jak z domova pohod...",https://zpravy.aktualne.cz/domaci/jak-se-spoji...,07-05-2020 22:38:53
101,r~02c1d02a8ed511eaa6f6ac1f6b220ee8,cesko-pozadalo-rusy-o-jednani-podle-smlouvy,2020-05-05,17:19:00,True,Opozice chtěla jednat o chování Ruska. Je to p...,"Ukázalo se jasně, kdo je kdo, kritizuje postoj...",https://zpravy.aktualne.cz/domaci/cesko-pozada...,07-05-2020 22:38:53
40,A200506_095011_brno-zpravy_krut,vychodni-populismus-panslovanstvi-propaganda-v...,2020-05-06,11:39:00,False,"Vědci se zaměří na panslovanský populismus, př...",Panslovanský nacionalistický populismus v posl...,https://www.idnes.cz/brno/zpravy/vychodni-popu...,07-05-2020 22:34:21
41,A200506_101930_liberec-zpravy_jape,hejtman-liberecky-kraj-martin-puta-soud-zavere...,2020-05-06,11:19:00,False,"Soud o úplatku spěje do finále, hejtman v závě...",Vzal hejtman Libereckého kraje Martin Půta úpl...,https://www.idnes.cz/liberec/zpravy/hejtman-li...,07-05-2020 22:34:21


In [19]:
articles = pd.read_csv(path_idnes)

In [20]:
### Headlines Analysis

In [21]:
# Word2vec model
import h2o
h2o.init()
from h2o.estimators.word2vec import H2OWord2vecEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "13.0.2" 2020-01-14; Java(TM) SE Runtime Environment (build 13.0.2+8); Java HotSpot(TM) 64-Bit Server VM (build 13.0.2+8, mixed mode, sharing)
  Starting server from /anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/c4/jxchjt990lg9002_rg92j8tr0000gn/T/tmphrky5mpo
  JVM stdout: /var/folders/c4/jxchjt990lg9002_rg92j8tr0000gn/T/tmphrky5mpo/h2o_simon_started_from_python.out
  JVM stderr: /var/folders/c4/jxchjt990lg9002_rg92j8tr0000gn/T/tmphrky5mpo/h2o_simon_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Prague
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.1
H2O_cluster_version_age:,1 month and 4 days
H2O_cluster_name:,H2O_from_python_simon_w4fsj3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [22]:
# Importing Aktualne.cz headlines
headlines_aktualne = pd.read_csv(path_aktualne, usecols=["headline"], encoding="utf-8")
headlines_aktualne.insert(0, "source", "Aktualne.cz")

# Importing iDnes.cz headlines
headlines_idnes = pd.read_csv(path_idnes, usecols=["headline"], encoding="utf-8")
headlines_idnes.insert(0, "source", "iDnes.cz")

# Concat headlines
df_headlines = pd.concat([headlines_aktualne, headlines_idnes], ignore_index=True)

# Convert pd.DataFrame to H2O DataFrame
headlines = h2o.H2OFrame(df_headlines)

# Check data
pd.concat([df_headlines[:5], df_headlines[-5:]])

Parse progress: |█████████████████████████████████████████████████████████| 100%


,source,headline
0,Aktualne.cz,"Další nouzový stav už nebude, ujistil Babiš. O..."
1,Aktualne.cz,"Uvolňování opatření je chaotické, míní Kubek. ..."
2,Aktualne.cz,"Běžte k lékařům, jinak nás čeká podzim rakovin..."
3,Aktualne.cz,Kolik je potvrzených nakažených v krajích? Akt...
4,Aktualne.cz,Za podnikatele zaplatíme minimálně třetinu náj...
3123,iDnes.cz,"David Černý navrhl most ve tvaru lokomotivy, s..."
3124,iDnes.cz,Současný způsob ochrany vlků je proti zdravému...
3125,iDnes.cz,Meteorologové varují před vichrem a mokrým sně...
3126,iDnes.cz,"Nádraží Vyšehrad chce Praha koupit, nebo vyvla..."
3127,iDnes.cz,"Obvinění manažeři MPSV do vazby nepůjdou, poli..."


In [23]:
# Defines the stopwords
STOP_WORDS = ["", "ačkoli","ahoj","ale","anebo","ano","asi","aspoň","během","bez","beze","blízko","bohužel","brzo","bude","budeme","budeš","budete","budou","budu","byl","byla","byli","bylo","byly","bys","čau","chce","chceme","chceš","chcete","chci","chtějí","chtít","chut\u0027","chuti","co","čtrnáct","čtyři","dál","dále","daleko","děkovat","děkujeme","děkuji","den","deset","devatenáct","devět","do","dobrý","docela","dva","dvacet","dvanáct","dvě","hodně","já","jak","jde","je","jeden","jedenáct","jedna","jedno","jednou","jedou","jeho","její","jejich","jemu","jen","jenom","ještě","jestli","jestliže","jí","jich","jím","jimi","jinak","jsem","jsi","jsme","jsou","jste","kam","kde","kdo","kdy","když","ke","kolik","kromě","která","které","kteří","který","kvůli","má","mají","málo","mám","máme","máš","máte","mé","mě","mezi","mí","mít","mně","mnou","moc","mohl","mohou","moje","moji","možná","můj","musí","může","my","na","nad","nade","nám","námi","naproti","nás","náš","naše","naši","ne","ně","nebo","nebyl","nebyla","nebyli","nebyly","něco","nedělá","nedělají","nedělám","neděláme","neděláš","neděláte","nějak","nejsi","někde","někdo","nemají","nemáme","nemáte","neměl","němu","není","nestačí","nevadí","než","nic","nich","ním","nimi","nula","od","ode","on","ona","oni","ono","ony","osm","osmnáct","pak","patnáct","pět","po","pořád","potom","pozdě","před","přes","přese","pro","proč","prosím","prostě","proti","protože","rovně","se","sedm","sedmnáct","šest","šestnáct","skoro","smějí","smí","snad","spolu","sta","sté","sto","ta","tady","tak","takhle","taky","tam","tamhle","tamhleto","tamto","tě","tebe","tebou","ted\u0027","tedy","ten","ti","tisíc","tisíce","to","tobě","tohle","toto","třeba","tři","třináct","trošku","tvá","tvé","tvoje","tvůj","ty","určitě","už","vám","vámi","vás","váš","vaše","vaši","ve","večer","vedle","vlastně","všechno","všichni","vůbec","vy","vždy","za","zač","zatímco","ze","že","aby","aj","ani","az","budem","budes","by","byt","ci","clanek","clanku","clanky","coz","cz","dalsi","design","dnes","email","ho","jako","jej","jeji","jeste","ji","jine","jiz","jses","kdyz","ktera","ktere","kteri","kterou","ktery","ma","mate","mi","mit","muj","muze","nam","napiste","nas","nasi","nejsou","neni","nez","nove","novy","pod","podle","pokud","pouze","prave","pred","pres","pri","proc","proto","protoze","prvni","pta","re","si","strana","sve","svych","svym","svymi","take","takze","tato","tema","tento","teto","tim","timto","tipy","toho","tohoto","tom","tomto","tomuto","tu","tuto","tyto","uz","vam","vas","vase","vice","vsak","zda","zde","zpet","zpravy","a","aniž","až","být","což","či","článek","článku","články","další","i","jenž","jiné","již","jseš","jšte","k","každý","kteři","ku","me","ná","napište","nechť","ní","nové","nový","o","práve","první","přede","při","s","sice","své","svůj","svých","svým","svými","také","takže","te","těma","této","tím","tímto","u","v","více","však","všechen","z","zpět","zprávy"]
import re
# Handles the tokenization
def tokenize(sentences, stop_word = STOP_WORDS):
    # Handlig Czech chracters, seperate by space instead of regex "\\W+"
    tokenized = sentences.tokenize(" ")
    tokenized = tokenized.rstrip()
    tokenized = tokenized.rstrip(".")
    tokenized = tokenized.rstrip(",")
    tokenized = tokenized.rstrip("!")
    tokenized = tokenized.rstrip("?")
    tokenized = tokenized.rstrip("-")

    tokenized_cleaned = tokenized
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [24]:
# Breaks down the string to sequence of words
words = tokenize(headlines["headline"])
words.describe

C1
nouzový
stav
nebude
ujistil
babiš
opatření
resortu
zdravotnictví
""
uvolňování


<bound method H2OFrame.describe of >

In [25]:
# Builds word2vec model
w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
w2v_model.train(training_frame=words)

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [26]:
w2v_model.find_synonyms("zeman", count = 10)

OrderedDict([('babiš', 0.9039709568023682),
             ('hamáček', 0.8778977990150452),
             ('vojtěch', 0.8263100981712341),
             ('zvažuje', 0.8150015473365784),
             ('otevřít', 0.7929820418357849),
             ('lánech', 0.7926300168037415),
             ('normálu', 0.7883449792861938),
             ('velikonoce', 0.7760518789291382),
             ('jednání', 0.7628975510597229),
             ('ohledně', 0.7525379657745361)])

In [27]:
# Function for prediction
def predict(headlines, w2v, gbm):
    words = tokenize(h2o.H2OFrame(headlines).ascharacter())
    headlines_vec = w2v.transform(words, aggregate_method="AVERAGE")
    print(gbm.predict(test_data=headlines_vec))

In [28]:
# Calculates a vector for each headline
headlines_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")

In [29]:
# Preparing train and validation data
valid_headlines = ~ headlines_vecs["C1"].isna()
data = headlines[valid_headlines,:].cbind(headlines_vecs[valid_headlines,:])
data_split = data.split_frame(ratios=[0.8])

# Builds GBM model
gbm_model = H2OGradientBoostingEstimator()
gbm_model.train(x = headlines_vecs.names,
                y="source", 
                training_frame = data_split[0], 
                validation_frame = data_split[1])

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [30]:
print("Predict!")
print(predict(["Justiční aktivista dostal pokutu 50 tisíc za databázi soudních jednání"], w2v_model, gbm_model))

Predict!
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,Aktualne.cz,iDnes.cz
iDnes.cz,0.689529,0.310471



None
